# <center> Mediterranean Model

In [47]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import networkx as nx
from pathlib import Path
from tqdm import tqdm
import pickle
import json

print("Imports loaded successfully!")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")

Imports loaded successfully!
NumPy version: 2.2.6
Pandas version: 2.3.3


In [48]:
# ==============================================================================
# GLOBAL VARIABLES - ADJUST THESE TO CONFIGURE THE SIMULATION
# ==============================================================================

# Simulation time parameters
INTERVAL_SIZE = 1/24  # Days per interval (1 hour)

# Ship-type-specific speeds (km/h)
SHIP_SPEEDS = {
    'tanker': 25,        # Tankers typically slower due to size/weight
    'bulk carrier': 28,  # Bulk carriers moderate speed
    'cargo ship': 32     # Container/cargo ships fastest
}

# Ship-type-specific port processing times (days)
PORT_LOADING_TIMES = {
    'tanker': 1.04,       # Tankers take longer to load (liquid cargo)
    'bulk carrier': 2.13, # Bulk carriers (based on UNCTAD)
    'cargo ship': 0.71    # Cargo ships faster loading (containerized)
}

PORT_UNLOADING_TIMES = {
    'tanker': 1.04,       # Tankers take longer to unload (liquid cargo)
    'bulk carrier': 2.13, # Bulk carriers moderate unloading time
    'cargo ship': 0.71   # Cargo ships faster unloading (containerized)
}

# Port capacity parameters (queuing theory)
TARGET_RHO = 0.8  # Target utilization factor (λ / (c × μ) < ρ)
                  # Lower values = less congestion, higher capacities
                  # Typical range: 0.7-0.9
MIN_PORT_CAPACITY = 1  # Minimum berths per port

# Data directories
NETWORK_DIR = '../part_3_network_extraction/network_outputs/'
SHIP_DATA_FILE = 'simulation_output_data/simulation_ship_data.csv'
OUTPUT_DIR = 'simulation_output_data/'

print("="*70)
print("GLOBAL VARIABLES CONFIGURED")
print("="*70)
print(f"Interval size: {INTERVAL_SIZE} days ({INTERVAL_SIZE * 24:.1f} hours)")
print(f"\nShip speeds (km/h):")
for ship_type, speed in SHIP_SPEEDS.items():
    print(f"  {ship_type.title()}: {speed} km/h ({speed * 24} km/day)")
print(f"\nPort loading times (days):")
for ship_type, time in PORT_LOADING_TIMES.items():
    print(f"  {ship_type.title()}: {time} days")
print(f"\nPort unloading times (days):")
for ship_type, time in PORT_UNLOADING_TIMES.items():
    print(f"  {ship_type.title()}: {time} days")
print(f"\nPort capacity parameters:")
print(f"  Target ρ (utilization): {TARGET_RHO}")
print(f"  Minimum capacity: {MIN_PORT_CAPACITY} berth(s)")
print("="*70)

GLOBAL VARIABLES CONFIGURED
Interval size: 0.041666666666666664 days (1.0 hours)

Ship speeds (km/h):
  Tanker: 25 km/h (600 km/day)
  Bulk Carrier: 28 km/h (672 km/day)
  Cargo Ship: 32 km/h (768 km/day)

Port loading times (days):
  Tanker: 1.04 days
  Bulk Carrier: 2.13 days
  Cargo Ship: 0.71 days

Port unloading times (days):
  Tanker: 1.04 days
  Bulk Carrier: 2.13 days
  Cargo Ship: 0.71 days

Port capacity parameters:
  Target ρ (utilization): 0.8
  Minimum capacity: 1 berth(s)


In [49]:
# Load the Contraction Hierarchies network
with open(f'{NETWORK_DIR}network_contraction_hierarchies.gpickle', 'rb') as f:
    G_ch = pickle.load(f)

print(f"Network loaded: {G_ch.number_of_nodes()} nodes, {G_ch.number_of_edges()} edges")

# Print node type breakdown
port_count = sum(1 for n in G_ch.nodes() if G_ch.nodes[n].get('source') == 'port')
choke_count = sum(1 for n in G_ch.nodes() if G_ch.nodes[n].get('source') == 'choke_point')
network_count = sum(1 for n in G_ch.nodes() if G_ch.nodes[n].get('source') not in ['port', 'choke_point'])

print(f"  Ports: {port_count}")
print(f"  Choke points: {choke_count}")
print(f"  Network nodes: {network_count}")

# Create port-country and country-port mappings
port_to_country = {}
country_to_ports = {}

for node in G_ch.nodes():
    if G_ch.nodes[node].get('source') == 'port':
        port_name = G_ch.nodes[node].get('portName')
        country = G_ch.nodes[node].get('country')
        
        if port_name and country:
            # Port to country mapping
            port_to_country[port_name] = country
            
            # Country to ports mapping
            if country not in country_to_ports:
                country_to_ports[country] = []
            country_to_ports[country].append(port_name)

print(f"\nPort-Country Mappings:")
print(f"  Unique ports: {len(port_to_country)}")
print(f"  Unique countries: {len(country_to_ports)}")

# Show sample
print(f"\nSample mappings:")
for port, country in list(port_to_country.items())[:5]:
    print(f"  {port} → {country}")

print(f"\nPorts by country:")
for country in sorted(country_to_ports.keys())[:10]:
    print(f"  {country}: {len(country_to_ports[country])} ports")

Network loaded: 328 nodes, 526 edges
  Ports: 42
  Choke points: 1
  Network nodes: 285

Port-Country Mappings:
  Unique ports: 42
  Unique countries: 22

Sample mappings:
  Tarragona → Spain
  Trieste → Italy
  Taranto → Italy
  Illichivsk → Ukraine
  Larnaca → Cyprus

Ports by country:
  Albania: 1 ports
  Algeria: 2 ports
  Bosnia And Herzegovina: 1 ports
  Croatia: 2 ports
  Cyprus: 1 ports
  Egypt: 3 ports
  France: 1 ports
  Greece: 1 ports
  Israel: 1 ports
  Italy: 12 ports


In [50]:
# Check if edges have 'length' parameter
print("Checking edge attributes...")
sample_edge = list(G_ch.edges(data=True))[0]
print(f"\nSample edge: {sample_edge[0]} -> {sample_edge[1]}")
print(f"Edge attributes: {sample_edge[2].keys()}")

# Check if 'length' exists
has_length = all('length' in G_ch[u][v] for u, v in G_ch.edges())
print(f"\nAll edges have 'length' attribute: {has_length}")

if has_length:
    lengths = [G_ch[u][v]['length'] for u, v in G_ch.edges()]
    print(f"Length statistics:")
    print(f"  Min: {min(lengths):.2f} km")
    print(f"  Max: {max(lengths):.2f} km")
    print(f"  Mean: {np.mean(lengths):.2f} km")
    print(f"  Total network length: {sum(lengths):.2f} km")
else:
    print("WARNING: Not all edges have 'length' attribute!")

Checking edge attributes...

Sample edge: 24 -> 2554
Edge attributes: dict_keys(['distance', 'source', 'length'])

All edges have 'length' attribute: True
Length statistics:
  Min: 6.83 km
  Max: 555.10 km
  Mean: 85.19 km
  Total network length: 44809.36 km


In [51]:
# ==============================================================================
# LOAD SHIP DATA FROM CSV
# ==============================================================================
print("="*70)
print("LOADING SHIP DATA FROM ship_generation.ipynb OUTPUT")
print("="*70)

# Load the ship data CSV generated by ship_generation.ipynb
ship_df = pd.read_csv(SHIP_DATA_FILE)

print(f"\n✓ Loaded {len(ship_df)} ships from {SHIP_DATA_FILE}")
print(f"  Columns: {len(ship_df.columns)}")
print(f"  Memory: {ship_df.memory_usage(deep=True).sum() / 1024:.1f} KB")

# Load HS codes mapping from JSON
with open('../../data/hs_codes_mapping.json', 'r') as f:
    hs_codes_mapping_full = json.load(f)

# Extract HS codes from ship_df columns
hs_codes_in_data = []
for col in ship_df.columns:
    if col.startswith('cargo_hs') and col.endswith('_weight'):
        hs_code = int(col.replace('cargo_hs', '').replace('_weight', ''))
        hs_codes_in_data.append(hs_code)

# Build HS_CODES dictionary with names from JSON
HS_CODES = {}
for hs_code in sorted(hs_codes_in_data):
    hs_str = str(hs_code).zfill(2)
    if hs_str in hs_codes_mapping_full:
        HS_CODES[hs_code] = hs_codes_mapping_full[hs_str]['name']
    else:
        HS_CODES[hs_code] = f"HS{hs_code:02d}" 

print(f"\nHS Codes in data: {list(HS_CODES.keys())}")

# Check if ship_type column exists
if 'ship_type' in ship_df.columns:
    print(f"\n✓ Ship type data found:")
    ship_type_counts = ship_df['ship_type'].value_counts()
    for ship_type, count in ship_type_counts.items():
        print(f"  {ship_type.title()}: {count} ships ({count/len(ship_df)*100:.1f}%)")
else:
    print(f"\n⚠ Warning: ship_type column not found - using default speed for all ships")

# Display statistics
print(f"\nShip cargo statistics:")
print(f"  Total weight: {ship_df['cargo_total_weight'].sum():,.0f} metric tons")
print(f"  Total value: ${ship_df['cargo_total_value'].sum():,.0f}")
print(f"  Mean weight: {ship_df['cargo_total_weight'].mean():,.0f} tons")
print(f"  Mean value: ${ship_df['cargo_total_value'].mean():,.0f}")
print(f"  Weight std: {ship_df['cargo_total_weight'].std():,.0f} tons")
print(f"  Value std: ${ship_df['cargo_total_value'].std():,.0f}")

# Get list of countries from ship data
countries_in_ships = sorted(set(ship_df['origin_country'].unique()) | set(ship_df['dest_country'].unique()))
print(f"\nCountries in ship data: {len(countries_in_ships)}")
print(f"  {countries_in_ships}")

print("="*70)

LOADING SHIP DATA FROM ship_generation.ipynb OUTPUT

✓ Loaded 6964 ships from simulation_output_data/simulation_ship_data.csv
  Columns: 198
  Memory: 11918.4 KB

HS Codes in data: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97]

✓ Ship type data found:
  Tanker: 3399 ships (48.8%)
  Bulk Carrier: 2119 ships (30.4%)
  Cargo Ship: 1446 ships (20.8%)

Ship cargo statistics:
  Total weight: 412,156,884 metric tons
  Total value: $651,216,077,865
  Mean weight: 59,184 tons
  Mean value: $93,511,786
  Weight std: 21,221 tons
  Value std: $260,490,857

Countries in ship data: 20
  ['Albania', 'Algeria', 'Croatia', 'Cyprus', 'Egypt', 'France', 'Greece', 'Israel', 'Italy'

In [52]:
# ==============================================================================
# COMPUTE ROUTES FOR SHIP COUNTRIES
# ==============================================================================
print("="*70)
print("COMPUTING ROUTES FOR COUNTRY PAIRS")
print("="*70)

# Create port name to node mapping
port_name_to_node = {}
for node in G_ch.nodes():
    if G_ch.nodes[node].get('source') == 'port':
        port_name = G_ch.nodes[node].get('portName')
        if port_name:
            port_name_to_node[port_name] = node

# Get unique country pairs from ship data
unique_pairs = set(zip(ship_df['origin_country'], ship_df['dest_country']))
print(f"\nFound {len(unique_pairs)} unique country pairs in ship data")

# Compute routes
country_pair_routes = {}

for origin_country, dest_country in tqdm(unique_pairs, desc="Computing routes"):
    origin_ports = country_to_ports.get(origin_country, [])
    dest_ports = country_to_ports.get(dest_country, [])
    
    if not origin_ports or not dest_ports:
        continue
    
    best_path = None
    best_length = float('inf')
    best_origin_port = None
    best_dest_port = None
    
    # Try all port combinations
    for o_port in origin_ports:
        for d_port in dest_ports:
            o_node = port_name_to_node.get(o_port)
            d_node = port_name_to_node.get(d_port)
            
            if o_node is None or d_node is None:
                continue
            
            try:
                path = nx.shortest_path(G_ch, o_node, d_node, weight='length')
                path_length = sum(G_ch[path[i]][path[i+1]].get('length', 0) 
                                for i in range(len(path)-1))
                
                if path_length < best_length:
                    best_length = path_length
                    best_path = path
                    best_origin_port = o_port
                    best_dest_port = d_port
            except (nx.NetworkXNoPath, nx.NodeNotFound):
                continue
    
    if best_path:
        country_pair_routes[(origin_country, dest_country)] = {
            'path': best_path,
            'length': best_length,
            'origin_port': best_origin_port,
            'dest_port': best_dest_port
        }

print(f"\n✓ Computed routes for {len(country_pair_routes)} country pairs")
print("="*70)

COMPUTING ROUTES FOR COUNTRY PAIRS

Found 230 unique country pairs in ship data


Computing routes: 100%|██████████| 230/230 [00:00<00:00, 1046.19it/s]


✓ Computed routes for 230 country pairs


In [53]:
# ==============================================================================
# RECONSTRUCT SHIP OBJECTS WITH ROUTES AND TIMING
# ==============================================================================
print("="*70)
print("RECONSTRUCTING SHIP OBJECTS WITH SHIP-TYPE-SPECIFIC PARAMETERS")
print("="*70)

# Calculate average ship weights for scaling
AVERAGE_SHIP_LOAD_WEIGHT = ship_df['cargo_total_weight'].mean()

# Calculate port processing time scaling factors for each ship type
scaling_factors = {}

for ship_type in ['tanker', 'bulk carrier', 'cargo ship']:
    # Get ships of this type
    type_mask = ship_df['ship_type'] == ship_type if 'ship_type' in ship_df.columns else ship_df.index.isin([])
    
    if type_mask.sum() > 0:
        type_weights = ship_df.loc[type_mask, 'cargo_total_weight']
        log_ratios = [np.log(1 + weight / AVERAGE_SHIP_LOAD_WEIGHT) for weight in type_weights]
        mean_log_ratio = np.mean(log_ratios)
        
        target_loading_intervals = PORT_LOADING_TIMES[ship_type] / INTERVAL_SIZE
        target_unloading_intervals = PORT_UNLOADING_TIMES[ship_type] / INTERVAL_SIZE
        
        loading_scale = target_loading_intervals / mean_log_ratio if mean_log_ratio > 0 else target_loading_intervals
        unloading_scale = target_unloading_intervals / mean_log_ratio if mean_log_ratio > 0 else target_unloading_intervals
    else:
        # Fallback if ship type not found
        loading_scale = PORT_LOADING_TIMES[ship_type] / INTERVAL_SIZE
        unloading_scale = PORT_UNLOADING_TIMES[ship_type] / INTERVAL_SIZE
    
    scaling_factors[ship_type] = {
        'loading_scale': loading_scale,
        'unloading_scale': unloading_scale
    }

print(f"Port processing scaling factors by ship type:")
for ship_type, factors in scaling_factors.items():
    print(f"  {ship_type.title()}:")
    print(f"    Loading scale: {factors['loading_scale']:.4f}")
    print(f"    Unloading scale: {factors['unloading_scale']:.4f}")

# Reconstruct ships with routes and timing
ships = []
skipped = 0

for idx, row in tqdm(ship_df.iterrows(), total=len(ship_df), desc="Reconstructing ships"):
    origin_country = row['origin_country']
    dest_country = row['dest_country']
    
    # Get route
    if (origin_country, dest_country) not in country_pair_routes:
        skipped += 1
        continue
    
    route_info = country_pair_routes[(origin_country, dest_country)]
    
    # Get ship type
    ship_type = row.get('ship_type', 'cargo ship')  # Default to cargo ship if not specified
    
    # Calculate loading/unloading times (WEIGHT-BASED, ship-type-specific)
    ship_weight = row['cargo_total_weight']
    log_ratio = np.log(1 + ship_weight / AVERAGE_SHIP_LOAD_WEIGHT)
    
    loading_mean = max(0.5, log_ratio * scaling_factors[ship_type]['loading_scale'])
    unloading_mean = max(0.5, log_ratio * scaling_factors[ship_type]['unloading_scale'])
    
    loading_intervals = max(1, np.random.poisson(loading_mean))
    unloading_intervals = max(1, np.random.poisson(unloading_mean))
    
    # Build ship dictionary
    ship = {
        'id': int(row['ship_id']),
        'origin_country': origin_country,
        'dest_country': dest_country,
        'origin_port': route_info['origin_port'],
        'dest_port': route_info['dest_port'],
        'ship_type': ship_type,  # ADD: ship type
        'path': route_info['path'],
        'path_length': route_info['length'],
        'cargo_total_weight': float(row['cargo_total_weight']),
        'cargo_total_value': float(row['cargo_total_value']),
        'loading_time': loading_intervals,
        'unloading_time': unloading_intervals,
        'loading_remaining': loading_intervals,
        'unloading_remaining': 0,
        'state': 'waiting_to_load',
        'current_edge_idx': 0,
        'km_into_current_edge': 0.0,
        'wait_intervals': 0,
        'completed': False
    }
    
    # Add HS code cargo fields
    for hs_code in HS_CODES.keys():
        ship[f'cargo_hs{hs_code}_weight'] = float(row[f'cargo_hs{hs_code}_weight'])
        ship[f'cargo_hs{hs_code}_value'] = float(row[f'cargo_hs{hs_code}_value'])
    
    ships.append(ship)

print(f"\n✓ Reconstructed {len(ships)} ships")
if skipped > 0:
    print(f"  ⚠ Skipped {skipped} ships (no route found)")

print(f"\nVerification:")
print(f"  Total cargo weight: {sum(s['cargo_total_weight'] for s in ships):,.0f} tons")
print(f"  Total cargo value: ${sum(s['cargo_total_value'] for s in ships):,.0f}")

# Show loading/unloading times by ship type
print(f"\nPort processing times by ship type:")
for ship_type in ['tanker', 'bulk carrier', 'cargo ship']:
    type_ships = [s for s in ships if s['ship_type'] == ship_type]
    if type_ships:
        mean_loading = np.mean([s['loading_time'] * INTERVAL_SIZE for s in type_ships])
        mean_unloading = np.mean([s['unloading_time'] * INTERVAL_SIZE for s in type_ships])
        print(f"  {ship_type.title()} ({len(type_ships)} ships):")
        print(f"    Mean loading: {mean_loading:.2f} days (target: {PORT_LOADING_TIMES[ship_type]})")
        print(f"    Mean unloading: {mean_unloading:.2f} days (target: {PORT_UNLOADING_TIMES[ship_type]})")

print("="*70)

RECONSTRUCTING SHIP OBJECTS WITH SHIP-TYPE-SPECIFIC PARAMETERS
Port processing scaling factors by ship type:
  Tanker:
    Loading scale: 39.7056
    Unloading scale: 39.7056
  Bulk Carrier:
    Loading scale: 64.3402
    Unloading scale: 64.3402
  Cargo Ship:
    Loading scale: 27.3347
    Unloading scale: 27.3347


Reconstructing ships: 100%|██████████| 6964/6964 [00:01<00:00, 3851.82it/s]


✓ Reconstructed 6964 ships

Verification:
  Total cargo weight: 412,156,884 tons
  Total cargo value: $651,216,077,865

Port processing times by ship type:
  Tanker (3399 ships):
    Mean loading: 1.04 days (target: 1.04)
    Mean unloading: 1.04 days (target: 1.04)
  Bulk Carrier (2119 ships):
    Mean loading: 2.12 days (target: 2.13)
    Mean unloading: 2.13 days (target: 2.13)
  Cargo Ship (1446 ships):
    Mean loading: 0.71 days (target: 0.71)
    Mean unloading: 0.71 days (target: 0.71)


In [54]:
# ==============================================================================
# INITIALIZE SIMULATION STATE
# ==============================================================================
print("="*70)
print("INITIALIZING SIMULATION STATE")
print("="*70)

# Port state tracking
port_states = {}
for node in G_ch.nodes():
    if G_ch.nodes[node].get('source') == 'port':
        port_name = G_ch.nodes[node].get('portName')
        if port_name:
            port_states[port_name] = {'loading': [], 'unloading': []}

print(f"✓ Initialized {len(port_states)} ports")

# Port wait statistics
port_wait_stats = {}
for port_name in port_states.keys():
    port_wait_stats[port_name] = {
        'total_wait_intervals': 0,
        'num_ships_waited': 0
    }

# Initialize tracking dictionaries
ship_edge_history = {}
ship_locations = {}
port_occupancy_by_timestep = {}

print(f"✓ Initialized tracking dictionaries")
print("="*70)

INITIALIZING SIMULATION STATE
✓ Initialized 42 ports
✓ Initialized tracking dictionaries


In [55]:
# ==============================================================================
# CALCULATE PORT CAPACITIES USING QUEUING THEORY
# ==============================================================================
print("="*70)
print("CALCULATING PORT CAPACITIES (QUEUING THEORY)")
print("="*70)

port_capacities = {}
port_statistics = {}

for port_name in port_states.keys():
    # Count ships using this port (both origin and destination)
    ships_from_port = sum(1 for s in ships if s['origin_port'] == port_name)
    ships_to_port = sum(1 for s in ships if s['dest_port'] == port_name)
    total_ships = ships_from_port + ships_to_port
    
    # Calculate arrival rate λ (ships per day)
    lambda_rate = total_ships / 365
    
    # Calculate average service time (weighted by ship types and operations)
    total_service_time = 0
    for s in ships:
        if s['origin_port'] == port_name:
            # Ships departing from this port need loading time
            total_service_time += s['loading_time'] * INTERVAL_SIZE
        if s['dest_port'] == port_name:
            # Ships arriving at this port need unloading time
            total_service_time += s['unloading_time'] * INTERVAL_SIZE
    
    avg_service_time = total_service_time / total_ships if total_ships > 0 else 1.0
    
    # Calculate service rate μ (ships per day per berth)
    mu_rate = 1 / avg_service_time if avg_service_time > 0 else 1.0
    
    # Calculate minimum capacity for target ρ
    # ρ = λ / (c × μ) < TARGET_RHO
    # c = λ / (μ × TARGET_RHO)
    c_min = lambda_rate / mu_rate / TARGET_RHO
    capacity = max(MIN_PORT_CAPACITY, int(np.ceil(c_min)))
    
    # Verify ρ < 1
    actual_rho = lambda_rate / (capacity * mu_rate) if capacity > 0 else 0
    
    port_capacities[port_name] = capacity
    port_statistics[port_name] = {
        'ships_from': ships_from_port,
        'ships_to': ships_to_port,
        'total_ships': total_ships,
        'lambda': lambda_rate,
        'avg_service_time': avg_service_time,
        'mu': mu_rate,
        'capacity': capacity,
        'rho': actual_rho
    }

print(f"✓ Calculated capacities for {len(port_capacities)} ports")
print(f"\nPort capacity statistics:")
print(f"  Min capacity: {min(port_capacities.values())}")
print(f"  Max capacity: {max(port_capacities.values())}")
print(f"  Mean capacity: {np.mean(list(port_capacities.values())):.1f}")
print(f"  Median capacity: {np.median(list(port_capacities.values())):.1f}")

# Verify all ρ < 1
max_rho = max(s['rho'] for s in port_statistics.values())
print(f"\nUtilization factor (ρ) statistics:")
print(f"  Target ρ: {TARGET_RHO}")
print(f"  Max ρ: {max_rho:.3f} (must be < 1)")
print(f"  Mean ρ: {np.mean([s['rho'] for s in port_statistics.values()]):.3f}")
if max_rho >= 1:
    print(f"  ⚠ WARNING: Some ports have ρ ≥ 1! System unstable!")
else:
    print(f"  ✓ All ports have ρ < 1 (stable queuing)")

# Show top 5 busiest ports
print(f"\nTop 5 busiest ports (by total ships):")
sorted_ports = sorted(port_statistics.items(), key=lambda x: x[1]['total_ships'], reverse=True)[:5]
for i, (port_name, stats) in enumerate(sorted_ports, 1):
    print(f"  {i}. {port_name}:")
    print(f"     Ships: {stats['total_ships']} (from: {stats['ships_from']}, to: {stats['ships_to']})")
    print(f"     λ: {stats['lambda']:.2f} ships/day, μ: {stats['mu']:.3f} ships/day/berth")
    print(f"     Capacity: {stats['capacity']} berths, ρ: {stats['rho']:.3f}")

print("="*70)


CALCULATING PORT CAPACITIES (QUEUING THEORY)
✓ Calculated capacities for 42 ports

Port capacity statistics:
  Min capacity: 1
  Max capacity: 9
  Mean capacity: 2.2
  Median capacity: 1.0

Utilization factor (ρ) statistics:
  Target ρ: 0.8
  Max ρ: 0.792 (must be < 1)
  Mean ρ: 0.349
  ✓ All ports have ρ < 1 (stable queuing)

Top 5 busiest ports (by total ships):
  1. Toulon:
     Ships: 2228 (from: 1025, to: 1203)
     λ: 6.10 ships/day, μ: 0.857 ships/day/berth
     Capacity: 9 berths, ρ: 0.792
  2. Barcelona:
     Ships: 1777 (from: 828, to: 949)
     λ: 4.87 ships/day, μ: 0.786 ships/day/berth
     Capacity: 8 berths, ρ: 0.774
  3. Genova:
     Ships: 1546 (from: 461, to: 1085)
     λ: 4.24 ships/day, μ: 0.943 ships/day/berth
     Capacity: 6 berths, ρ: 0.748
  4. Alger:
     Ships: 851 (from: 771, to: 80)
     λ: 2.33 ships/day, μ: 0.866 ships/day/berth
     Capacity: 4 berths, ρ: 0.673
  5. Novorossiysk:
     Ships: 661 (from: 636, to: 25)
     λ: 1.81 ships/day, μ: 0.680 ships/

In [56]:
# ==============================================================================
# RUN SIMULATION - WEIGHT AND VALUE TRACKING (VERSION 3)
# ==============================================================================
# This version tracks BOTH weight and value for all cargo
print("Running Mediterranean shipping simulation with WEIGHT AND VALUE tracking...")
print(f"Simulating 365 days with {INTERVAL_SIZE}-day intervals")
print("VERSION 3: Weight-based simulation with value tracking")
print("="*70)

# Helper function to normalize edge keys
def normalize_edge_key(node1, node2):
    """Return canonical edge key for undirected graphs"""
    try:
        if node1 < node2:
            return (node1, node2)
        else:
            return (node2, node1)
    except TypeError:
        if str(node1) < str(node2):
            return (node1, node2)
        else:
            return (node2, node1)

# Initialize edge traffic data structure with WEIGHT AND VALUE fields
def create_edge_traffic_entry():
    """Create edge traffic entry with weight and value fields for each HS code"""
    entry = {
        'ship_count': 0,
        'origin_dest_pairs': {},
        'cargo_total_weight': 0.0,  # NEW: total weight
        'cargo_total_value': 0.0,   # NEW: total value
        'total_time_hours': 0.0
    }
    # Add weight and value fields for each HS code
    for hs_code in HS_CODES.keys():
        entry[f'cargo_hs{hs_code}_weight'] = 0.0
        entry[f'cargo_hs{hs_code}_value'] = 0.0
    return entry

def create_od_pair_entry():
    """Create OD pair entry with weight and value fields"""
    entry = {
        'count': 0,
        'cargo_total_weight': 0.0,
        'cargo_total_value': 0.0
    }
    # Add weight and value fields for each HS code
    for hs_code in HS_CODES.keys():
        entry[f'cargo_hs{hs_code}_weight'] = 0.0
        entry[f'cargo_hs{hs_code}_value'] = 0.0
    return entry

# Initialize ALL network edges
edge_traffic = {}
for node1, node2 in G_ch.edges():
    edge_key = normalize_edge_key(node1, node2)
    if edge_key not in edge_traffic:
        edge_traffic[edge_key] = create_edge_traffic_entry()

print(f"Initialized edge_traffic with {len(edge_traffic)} network edges")

# Parameters
lambda_param = len(ships) * INTERVAL_SIZE / 365
hours_per_interval = INTERVAL_SIZE * 24

# Simulation state
simulation_steps = []
n_intervals = int(365 / INTERVAL_SIZE)

# Statistics
total_loading_intervals = 0
total_unloading_intervals = 0
num_ships_completed = 0
cargo_additions = 0

# Track total cargo transported (not per-edge, just per-ship)
total_cargo_weight_transported = 0
total_cargo_value_transported = 0

# Main simulation loop
for interval in tqdm(range(n_intervals), desc="Simulating days"):
    day = interval * INTERVAL_SIZE
    n_new_ships = np.random.poisson(lambda_param)

    # Add new ships
    ships_to_add = []
    for _ in range(n_new_ships):
        if ships:
            new_ship = ships.pop(0)
            ships_to_add.append(new_ship)

    if interval == 0:
        active_ships = ships_to_add
    else:
        active_ships.extend(ships_to_add)

    current_locations = {}

    # === PHASE 1: Port operations ===
    for ship in active_ships:
        if ship['state'] == 'waiting_to_load':
            port = ship['origin_port']
            if len(port_states[port]['loading']) < port_capacities[port]:
                port_states[port]['loading'].append(ship['id'])
                ship['state'] = 'loading'
            else:
                ship['wait_intervals'] += 1

        elif ship['state'] == 'loading':
            ship['loading_remaining'] -= 1
            if ship['loading_remaining'] <= 0:
                port = ship['origin_port']
                port_states[port]['loading'].remove(ship['id'])
                ship['state'] = 'traveling'
                total_loading_intervals += ship['loading_time']

        elif ship['state'] == 'waiting_to_unload':
            port = ship['dest_port']
            if len(port_states[port]['unloading']) < port_capacities[port]:
                port_states[port]['unloading'].append(ship['id'])
                ship['state'] = 'unloading'
                ship['unloading_remaining'] = ship['unloading_time']
            else:
                ship['wait_intervals'] += 1

        elif ship['state'] == 'unloading':
            ship['unloading_remaining'] -= 1
            if ship['unloading_remaining'] <= 0:
                port = ship['dest_port']
                port_states[port]['unloading'].remove(ship['id'])
                ship['state'] = 'completed'
                ship['completed'] = True
                total_cargo_weight_transported += ship["cargo_total_weight"]
                total_cargo_value_transported += ship["cargo_total_value"]
                total_unloading_intervals += ship['unloading_time']
                num_ships_completed += 1

    # Record port locations
    for ship in active_ships:
        if ship['state'] in ['waiting_to_load', 'loading']:
            current_locations[ship['id']] = {
                'status': 'loading',
                'port': ship['origin_port']
            }
        elif ship['state'] in ['waiting_to_unload', 'unloading']:
            current_locations[ship['id']] = {
                'status': 'unloading',
                'port': ship['dest_port']
            }

    # === PHASE 2: Move traveling ships ===
    ships_to_remove = []

    for ship in active_ships:
        if ship['state'] != 'traveling':
            continue

        if ship['completed']:
            ships_to_remove.append(ship)
            continue

        km_remaining = SHIP_SPEEDS[ship['ship_type']] * 24 * INTERVAL_SIZE
        edge_time_allocation = {}

        while km_remaining > 0 and ship['state'] == 'traveling':
            edge_idx = ship['current_edge_idx']
            if edge_idx >= len(ship['path']) - 1:
                ship['state'] = 'waiting_to_unload'
                current_locations[ship['id']] = {
                    'status': 'arrived',
                    'port': ship['dest_port']
                }
                break

            node1 = ship['path'][edge_idx]
            node2 = ship['path'][edge_idx + 1]

            # Get edge length
            if G_ch.has_edge(node1, node2):
                edge_length = G_ch[node1][node2].get('length', 0)
            elif G_ch.has_edge(node2, node1):
                edge_length = G_ch[node2][node1].get('length', 0)
            else:
                edge_length = 0

            km_left_in_edge = edge_length - ship['km_into_current_edge']
            edge_key = normalize_edge_key(node1, node2)
            ship_id_int = ship['id']

            if ship_id_int not in ship_edge_history:
                ship_edge_history[ship_id_int] = set()

            # CARGO ATTRIBUTION: Add weight and value when ship enters edge
            if edge_key not in ship_edge_history[ship_id_int]:
                ship_edge_history[ship_id_int].add(edge_key)

                if edge_key not in edge_traffic:
                    edge_traffic[edge_key] = create_edge_traffic_entry()

                edge_traffic[edge_key]['ship_count'] += 1
                edge_traffic[edge_key]['cargo_total_weight'] += ship['cargo_total_weight']
                edge_traffic[edge_key]['cargo_total_value'] += ship['cargo_total_value']

                # Add weight and value for each HS code
                for hs_code in HS_CODES.keys():
                    weight_field = f'cargo_hs{hs_code}_weight'
                    value_field = f'cargo_hs{hs_code}_value'
                    edge_traffic[edge_key][weight_field] += ship.get(weight_field, 0.0)
                    edge_traffic[edge_key][value_field] += ship.get(value_field, 0.0)

                # Track OD pairs
                od_key = (ship['origin_country'], ship['dest_country'])
                if od_key not in edge_traffic[edge_key]['origin_dest_pairs']:
                    edge_traffic[edge_key]['origin_dest_pairs'][od_key] = create_od_pair_entry()

                edge_traffic[edge_key]['origin_dest_pairs'][od_key]['count'] += 1
                edge_traffic[edge_key]['origin_dest_pairs'][od_key]['cargo_total_weight'] += ship['cargo_total_weight']
                edge_traffic[edge_key]['origin_dest_pairs'][od_key]['cargo_total_value'] += ship['cargo_total_value']

                # Add OD weight and value for each HS code
                for hs_code in HS_CODES.keys():
                    weight_field = f'cargo_hs{hs_code}_weight'
                    value_field = f'cargo_hs{hs_code}_value'
                    edge_traffic[edge_key]['origin_dest_pairs'][od_key][weight_field] += ship.get(weight_field, 0.0)
                    edge_traffic[edge_key]['origin_dest_pairs'][od_key][value_field] += ship.get(value_field, 0.0)

                cargo_additions += 1

            if km_remaining >= km_left_in_edge:
                # Complete edge
                km_traveled = km_left_in_edge
                hours_for_segment = (km_traveled / SHIP_SPEEDS[ship['ship_type']]) if SHIP_SPEEDS[ship['ship_type']] > 0 else 0

                if edge_key not in edge_time_allocation:
                    edge_time_allocation[edge_key] = 0.0
                edge_time_allocation[edge_key] += hours_for_segment

                km_remaining -= km_left_in_edge
                ship['current_edge_idx'] += 1
                ship['km_into_current_edge'] = 0.0
            else:
                # Partial edge
                km_traveled = km_remaining
                hours_for_segment = (km_traveled / SHIP_SPEEDS[ship['ship_type']]) if SHIP_SPEEDS[ship['ship_type']] > 0 else 0

                if edge_key not in edge_time_allocation:
                    edge_time_allocation[edge_key] = 0.0
                edge_time_allocation[edge_key] += hours_for_segment

                ship['km_into_current_edge'] += km_remaining

                current_locations[ship['id']] = {
                    'status': 'active',
                    'edge': [str(node1), str(node2)],
                    'edge_length_km': float(edge_length),
                    'progress_km': float(ship['km_into_current_edge']),
                    'progress_fraction': float(ship['km_into_current_edge'] / edge_length) if edge_length > 0 else 0.0
                }

                km_remaining = 0

        # Add time to edges
        for edge_key, hours_spent in edge_time_allocation.items():
            if edge_key in edge_traffic:
                edge_traffic[edge_key]['total_time_hours'] += hours_spent

    # Remove completed ships
    for ship in ships_to_remove:
        active_ships.remove(ship)

    # Record port occupancy
    port_occupancy_by_timestep[interval] = {}
    for port_name, state in port_states.items():
        num_ships = len(state['loading']) + len(state['unloading'])
        if num_ships > 0:
            port_occupancy_by_timestep[interval][port_name] = num_ships

    # Store locations
    if current_locations:
        ship_locations[str(day)] = current_locations

    simulation_steps.append({
        'day': day,
        'active_ships': len(active_ships),
        'new_ships': n_new_ships,
        'completed_ships': len(ships_to_remove)
    })

# Calculate wait stats
for ship in active_ships:
    if ship['wait_intervals'] > 0:
        if ship['state'] in ['waiting_to_load', 'loading']:
            port = ship['origin_port']
        else:
            port = ship['dest_port']
        port_wait_stats[port]['total_wait_intervals'] += ship['wait_intervals']
        port_wait_stats[port]['num_ships_waited'] += 1

print(f"\n{'='*70}")
print(f"SIMULATION COMPLETE!")
print(f"{'='*70}")
print(f"Total edges: {len(edge_traffic)} (network: {G_ch.number_of_edges()})")
print(f"Ship-edge traversals: {cargo_additions}")
print(f"Ships completed: {num_ships_completed}")
print(f"Timesteps: {len(ship_locations)}")

edges_with_traffic = sum(1 for e in edge_traffic.values() if e['ship_count'] > 0)
print(f"\nEdge coverage:")
print(f"  With traffic: {edges_with_traffic}")
print(f"  Without traffic: {len(edge_traffic) - edges_with_traffic}")

# Show WEIGHT breakdown
print(f"\nTraffic by commodity (WEIGHT in metric tons):")
for hs_code, description in HS_CODES.items():
    weight_field = f'cargo_hs{hs_code}_weight'
    total_weight = sum(e[weight_field] for e in edge_traffic.values())
    print(f"  HS{hs_code:02d} ({description}): {total_weight:,.0f} tons")

# Show VALUE breakdown  
print(f"\nTraffic by commodity (VALUE in USD):")
for hs_code, description in HS_CODES.items():
    value_field = f'cargo_hs{hs_code}_value'
    total_value = sum(e[value_field] for e in edge_traffic.values())
    print(f"  HS{hs_code:02d} ({description}): ${total_value:,.0f}")

# Time statistics
total_time = sum(e['total_time_hours'] for e in edge_traffic.values())
print(f"\nTime statistics:")
print(f"  Total ship-hours: {total_time:,.0f} hours")
print(f"  Total ship-days: {total_time / 24:,.0f} days")
print(f"  Avg per traversal: {total_time / cargo_additions:.2f} hours" if cargo_additions > 0 else "  N/A")

# Port statistics
avg_loading = total_loading_intervals * INTERVAL_SIZE / num_ships_completed if num_ships_completed > 0 else 0
avg_unloading = total_unloading_intervals * INTERVAL_SIZE / num_ships_completed if num_ships_completed > 0 else 0

print(f"\nPort processing:")
print(f"  Avg loading: {avg_loading:.2f} days")
print(f"  Avg unloading: {avg_unloading:.2f} days")

total_wait = sum(s['total_wait_intervals'] for s in port_wait_stats.values())
total_waited = sum(s['num_ships_waited'] for s in port_wait_stats.values())
print(f"\nWait statistics:")
print(f"  Total wait: {total_wait * INTERVAL_SIZE:.2f} days")
print(f"  Ships waited: {total_waited}")
print(f"  Avg wait: {total_wait / total_waited * INTERVAL_SIZE:.2f} days" if total_waited > 0 else "  None")

print(f"{'='*70}")

Running Mediterranean shipping simulation with WEIGHT AND VALUE tracking...
Simulating 365 days with 0.041666666666666664-day intervals
VERSION 3: Weight-based simulation with value tracking
Initialized edge_traffic with 526 network edges


Simulating days: 100%|██████████| 8760/8760 [00:18<00:00, 472.29it/s]


SIMULATION COMPLETE!
Total edges: 526 (network: 526)
Ship-edge traversals: 88778
Ships completed: 6934
Timesteps: 8760

Edge coverage:
  With traffic: 340
  Without traffic: 186

Traffic by commodity (WEIGHT in metric tons):
  HS01 (Live animals): 7,538,889 tons
  HS02 (Meat and edible meat offal): 17,226,994 tons
  HS03 (Fish and crustaceans, molluscs and other aquatic invertebrates): 7,409,316 tons
  HS04 (Dairy produce; birds eggs; natural honey; edible products of animal origin): 14,202,282 tons
  HS05 (Products of animal origin, not elsewhere specified or included): 1,591,729 tons
  HS06 (Live trees and other plants; bulbs, roots and the like; cut flowers): 1,289,202 tons
  HS07 (Edible vegetables and certain roots and tubers): 73,698,757 tons
  HS08 (Edible fruit and nuts; peel of citrus fruit or melons): 51,433,599 tons
  HS09 (Coffee, tea, mate and spices): 3,975,316 tons
  HS10 (Cereals): 1,577,457,076 tons
  HS11 (Products of the milling industry; malt; starches; inulin; whe

In [57]:
# ==============================================================================
# EXPORT SIMULATION RESULTS (SHIP LOCATIONS, EDGE STATS, PORT OCCUPANCY)
# ==============================================================================
print("="*70)
print("EXPORTING SIMULATION RESULTS")
print("="*70)

# Note: simulation_ship_data.csv is already exported by ship_generation.ipynb
# This cell exports the remaining 3 output files

# 1. Export simulation_ship_location.json
print(f"\n1. Exporting ship locations...")
with open('simulation_output_data/simulation_ship_location.json', 'w') as f:
    json.dump(ship_locations, f, indent=2)
print(f"   ✓ Exported simulation_ship_location.json ({len(ship_locations)} timesteps)")

# 2. Export simulation_edge_statistics.csv WITH WEIGHT AND VALUE
print(f"\n2. Exporting edge statistics...")
edge_stats_list = []

for edge_key, traffic_data in edge_traffic.items():
    node1, node2 = edge_key
    
    # Get edge length
    edge_length = 0
    if G_ch.has_edge(node1, node2):
        edge_length = G_ch[node1][node2].get('length', 0)
    elif G_ch.has_edge(node2, node1):
        edge_length = G_ch[node2][node1].get('length', 0)
    
    edge_stat = {
        'node1': str(node1),
        'node2': str(node2),
        'edge_length_km': float(edge_length),
        'ship_count': int(traffic_data['ship_count']),
        'total_time_hours': float(traffic_data['total_time_hours']),
        'cargo_total_weight': float(traffic_data['cargo_total_weight']),
        'cargo_total_value': float(traffic_data['cargo_total_value'])
    }
    
    # Add weight and value for each HS code
    for hs_code in HS_CODES.keys():
        weight_field = f'cargo_hs{hs_code}_weight'
        value_field = f'cargo_hs{hs_code}_value'
        edge_stat[weight_field] = float(traffic_data[weight_field])
        edge_stat[value_field] = float(traffic_data[value_field])
    
    edge_stats_list.append(edge_stat)

edge_df = pd.DataFrame(edge_stats_list)
edge_df.to_csv('simulation_output_data/simulation_edge_statistics.csv', index=False)
print(f"   ✓ Exported simulation_edge_statistics.csv ({len(edge_df)} edges)")

# Statistics
print(f"\n   Edge statistics:")
print(f"     Edges with traffic: {(edge_df['ship_count'] > 0).sum()}")
print(f"     Edges with NO traffic: {(edge_df['ship_count'] == 0).sum()}")
print(f"     Total weight: {edge_df['cargo_total_weight'].sum():,.0f} tons")
print(f"     Total value: ${edge_df['cargo_total_value'].sum():,.0f}")

# 3. Export simulation_port_occupancy.csv
print(f"\n3. Exporting port occupancy...")
port_occupancy_list = []
n_intervals = int(365 / INTERVAL_SIZE)

for interval in range(n_intervals):
    timestep_day = interval * INTERVAL_SIZE
    
    if interval in port_occupancy_by_timestep:
        for port_name, num_ships in port_occupancy_by_timestep[interval].items():
            port_occupancy_list.append({
                'timestep': interval,
                'day': float(timestep_day),
                'port_name': port_name,
                'num_ships': int(num_ships),
                'capacity': int(port_capacities[port_name])
            })

port_occ_df = pd.DataFrame(port_occupancy_list)
port_occ_df.to_csv('simulation_output_data/simulation_port_occupancy.csv', index=False)
print(f"   ✓ Exported simulation_port_occupancy.csv ({len(port_occ_df)} records)")

print(f"\n{'='*70}")
print("SIMULATION OUTPUT COMPLETE!")
print("="*70)
print(f"\nOutput files:")
print(f"  1. simulation_ship_data.csv (from ship_generation.ipynb)")
print(f"     - Ship cargo composition with weight and value")
print(f"")
print(f"  2. simulation_ship_location.json")
print(f"     - Ship positions at each timestep")
print(f"")
print(f"  3. simulation_edge_statistics.csv")
print(f"     - ALL {len(edge_df)} network edges with traffic statistics")
print(f"     - Includes BOTH weight (tons) and value (USD)")
print(f"")
print(f"  4. simulation_port_occupancy.csv")
print(f"     - Port occupancy over time")
print(f"")
print("="*70)
print("\nNext steps:")
print("  - Use Video_Simulation.ipynb to create animated videos")
print("  - Use Simulation_Visualizations.ipynb for static visualizations")
print("="*70)

EXPORTING SIMULATION RESULTS

1. Exporting ship locations...
   ✓ Exported simulation_ship_location.json (8760 timesteps)

2. Exporting edge statistics...
   ✓ Exported simulation_edge_statistics.csv (526 edges)

   Edge statistics:
     Edges with traffic: 340
     Edges with NO traffic: 186
     Total weight: 5,421,621,610 tons
     Total value: $7,431,294,432,629

3. Exporting port occupancy...
   ✓ Exported simulation_port_occupancy.csv (161814 records)

SIMULATION OUTPUT COMPLETE!

Output files:
  1. simulation_ship_data.csv (from ship_generation.ipynb)
     - Ship cargo composition with weight and value

  2. simulation_ship_location.json
     - Ship positions at each timestep

  3. simulation_edge_statistics.csv
     - ALL 526 network edges with traffic statistics
     - Includes BOTH weight (tons) and value (USD)

  4. simulation_port_occupancy.csv
     - Port occupancy over time


Next steps:
  - Use Video_Simulation.ipynb to create animated videos
  - Use Simulation_Visualiza

In [58]:
# ==============================================================================
# SIMULATION SUMMARY
# ==============================================================================
print("="*70)
print("MEDITERRANEAN SHIPPING SIMULATION - SUMMARY")
print("="*70)

print(f"\nSimulation Parameters:")
print(f"  Ships simulated: {num_ships_completed}")
print(f"  Interval size: {INTERVAL_SIZE} days ({INTERVAL_SIZE * 24:.1f} hours)")
print(f"  Simulation period: 365 days (1 year)")

print(f"\nCargo Statistics:")
print(f"  Total weight transported: {total_cargo_weight_transported:,.0f} metric tons")
print(f"  Total value transported: ${total_cargo_value_transported:,.0f}")

print(f"\nNetwork Statistics:")
print(f"  Total network edges: {G_ch.number_of_edges()}")
print(f"  Edges with traffic: {sum(1 for e in edge_traffic.values() if e['ship_count'] > 0)}")
print(f"  Ship-edge traversals: {sum(e['ship_count'] for e in edge_traffic.values())}")
print(f"  Timesteps tracked: {len(ship_locations)}")

# Find busiest routes BY WEIGHT
print(f"\nTop 5 Busiest Edges (by cargo WEIGHT):")
sorted_edges_weight = sorted(edge_traffic.items(), 
                             key=lambda x: x[1]['cargo_total_weight'], 
                             reverse=True)[:5]

for i, (edge, data) in enumerate(sorted_edges_weight, 1):
    node1, node2 = edge
    name1 = G_ch.nodes[node1].get('portName', f"Node {node1}")
    name2 = G_ch.nodes[node2].get('portName', f"Node {node2}")
    
    print(f"  {i}. {name1} ↔ {name2}")
    print(f"     Ships: {data['ship_count']:,} | Weight: {data['cargo_total_weight']:,.0f} tons")

print(f"\nOutput Files:")
print(f"  ✓ simulation_ship_data.csv (from ship_generation.ipynb)")
print(f"  ✓ simulation_ship_location.json")
print(f"  ✓ simulation_edge_statistics.csv")
print(f"  ✓ simulation_port_occupancy.csv")

print(f"\n{'='*70}")
print("WEIGHT-BASED SIMULATION COMPLETE!")
print("="*70)
print("\nWorkflow:")
print("  1. ship_generation.ipynb - Generate ships with weight/value data")
print("  2. Mediterranean_Model.ipynb - Simulate movement and generate outputs")
print()
print("Key Features:")
print("  ✓ Weight-based ship sampling and routing")
print("  ✓ Port queuing with dynamic capacities (queuing theory)")
print("  ✓ Ship-type-specific speeds and processing times")
print("  ✓ Both weight (tons) and value (USD) tracked")
print("  ✓ Modular design for faster iteration")
print("="*70)

MEDITERRANEAN SHIPPING SIMULATION - SUMMARY

Simulation Parameters:
  Ships simulated: 6934
  Interval size: 0.041666666666666664 days (1.0 hours)
  Simulation period: 365 days (1 year)

Cargo Statistics:
  Total weight transported: 409,964,378 metric tons
  Total value transported: $649,968,241,548

Network Statistics:
  Total network edges: 526
  Edges with traffic: 340
  Ship-edge traversals: 88778
  Timesteps tracked: 8760

Top 5 Busiest Edges (by cargo WEIGHT):
  1. Node 4927 ↔ Istanbul
     Ships: 1,559 | Weight: 100,918,442 tons
  2. Node 834 ↔ Node 1662
     Ships: 1,558 | Weight: 100,835,290 tons
  3. Node 834 ↔ Node 4927
     Ships: 1,558 | Weight: 100,835,290 tons
  4. Node 1662 ↔ Node 7634
     Ships: 1,557 | Weight: 100,782,782 tons
  5. Node 3516 ↔ Node 8313
     Ships: 1,546 | Weight: 84,665,603 tons

Output Files:
  ✓ simulation_ship_data.csv (from ship_generation.ipynb)
  ✓ simulation_ship_location.json
  ✓ simulation_edge_statistics.csv
  ✓ simulation_port_occupancy.c